In [1]:
#!pip install psycopg2

In [1]:
import psycopg2
import pandas as pd
import datetime as dt
import os

import sqlalchemy as sqa
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import glob 


In [2]:
#in the next few kernels we will connect to PostGRES and create a new DB
# Connect to PostgreSQL DBMS
database = 'nba2018db'
user = 'postgres'
password ='postgres' #needs password

con = psycopg2.connect("user=postgres password='postgres'");
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);

In [3]:
# Obtain a DB Cursor
cursor = con.cursor();
name_Database = "nba2018db";

In [4]:
# Create table statement
sqlCreateDatabase = "create database "+name_Database+";"

In [5]:
# Create a table in PostgreSQL database and then close the connection
cursor.execute(sqlCreateDatabase);
# con.close()

In [6]:
# connecting to the new database
# a second cursor was created so that it would connect properly to the new db

con = psycopg2.connect(database='nba2018db', user=user, password=password)
cursor2 = con.cursor()

In [7]:
filepath = os.path.join('sql','NBA2018.sql')


In [8]:
#using the export of the ERD diagram from https://app.quickdatabasediagrams.com/#/
#to create the tables in the nba2018db

sql_file = open(filepath,'r', encoding='utf-8-sig')
cursor2.execute(sql_file.read())

In [33]:
#create an iterable list of our inputs from the directory

directory_path = os.path.join('..','outputs','*.csv')
file_list = []

for fname in glob.glob(directory_path):
    file_list.append(fname)

file_list
table_list = []

for i in range(len(file_list)):
    table_list.append(file_list[i].replace('_cleaned.csv','').replace('../outputs/', ''))

file_list = sorted(file_list)
table_list =sorted(table_list)

for i in range(len(table_list)):
    table_list[i] = table_list[i][2:]
    

print(table_list)  

['players', 'teams', 'players_team', 'salary', 'combine', 'games', 'stats']


In [31]:
# import csv
# with open(file_list[0], 'r') as f:
#     reader = csv.reader(f)
#     next(reader) # Skip the header row.
#     for row in reader:
#         cursor2.execute(
#         f"INSERT INTO {table_list[0]} VALUES ({row})",
#         row
#     )

In [32]:
#ingest all of the CSV file into the database using an for-loop
#a null='NULL' statement is required so that any empty field 
#is ingested properly

for i in range(len(file_list)):
#     filepath = os.path.join('outputs','teams_cleaned.csv')
    with open(file_list[i], 'r') as f:
        next(f) # Skip the header row.
        cursor2.copy_from(f, table_list[i], sep=',', null='NULL')
        con.commit()

UndefinedTable: relation "a_players" does not exist


In [13]:
con.close()
cursor2.close()
cursor.close()